# Tutorial 11292021: Pandas DataFrames (and a bit of file I/O)
[The official project homepage](https://pandas.pydata.org)

* Goal
    * Extend what we learned about Series objects in the previous tutorial to their 2D counterpart - DataFrames
    * Develop some tools for dealing with missing data (not exhaustive, but a good start)
    * Take this chance to also learn a bit about file input/output (I/O) and some other more advanced coding techniques

## DataFrames

[Pandas quick start guide for DataFrames](https://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe)

* A DataFrame (DF) is a labeled data struture that can be thought of as a 2D extension of the Series objects that we discussed in the first part of the tutorial
* A DF can accept many types of input, from a 2D ndarray, multiple Series, a dict of 1D arrays, another DF, etc
* Like a Series, DFs contain data values and their labels. Because we're now dealing with a 2D structure, we call the **row labels the index argument** and the **column labels the column argument**. 
    * Like a Series, if you don't explicitly assign row and column labels, then they will be auto-generated (but not as useful as specifying the labels yourself!)

<div class="alert alert-info">
Much of what we learned about Series objects will generalize to DFs, so here we'll focus on some of key functionality that might not be obvious based on the previous tutorial.
</div>

<div class="alert alert-info">
One more quick note: if using an older version of Python (earlier than 3.6) and Pandas (earlier than 0.23) and you create a DF from a dict without explicitly specifying column names, then the column names will be entered into the DF based on lexical order
</div>

## Import libs

In [1]:
# standard numpy module
import numpy as np

# import a generic pandas object and also a few specific functions that we'll use
import pandas as pd 
from pandas import DataFrame, read_csv

# new - get and store current file path for file i/o later on in tutorial
import os
cwd = os.getcwd()

## Make up a data set to demonstrate functionality, will import some real data later on
* Here we'll pretend that we did a unit recording experiment 
    * There are two stimulus conditions
    * And we are recording from 10 different neurons 

In [2]:
# seed random number generator so that we're all seeing the same thing in class
np.random.seed(0)

In [3]:
# index lables for our 10 neurons...see previous tutorial for more elegant ways of generating
# index labels, here we're just going to write them out
neuron_labels = ['Nrn0', 'Nrn1','Nrn2','Nrn3','Nrn4']  

In [4]:
# generate a response to each of two stimuli...use random.randint
min_resp = 0  # inclusive
max_resp = 90 # exclusive

# generate response in each neuron to stimulus 1...
resp1_hz = np.random.randint(min_resp, max_resp, len(neuron_labels))

# generate response in each neuron to stimulus 2...
resp2_hz = np.random.randint(min_resp, max_resp, len(neuron_labels))

## New - use 'zip' function to wrap up the data from each list into one list
[reference page for zip](https://www.w3schools.com/python/ref_func_zip.asp)

* Operates just like it sounds  - takes a set of iterators and groups them together into a single iterator with the 1st element in the resultant iterator comprised of the first element of each iterator 'zipped' together, then the second element from each iterator zipped together, etc. 
* Length of resulting iterator limited by the length of the shortest input iterator!

<div class="alert alert-warning">
Because the length of the resulting iterator is limited by length of shortest input iterator, you can sometimes not get an error if you try to zip together iterators with unequal lengths - this is fine if intenitonal, but if the unequal length was caused by a bug, then you may not find it when using zip!
</div>

In [5]:
neuron_data = list(zip(resp1_hz, resp2_hz))
print(neuron_data)

print('Grab one index to see the two response arrays zipped together:')
print(neuron_data[4])

[(44, 9), (47, 83), (64, 21), (67, 36), (67, 87)]
Grab one index to see the two response arrays zipped together:
(67, 87)


In [6]:
# note: to unzip go like this and you'll get back the original
uz_data1, uz_data2 = zip(*neuron_data)
print(uz_data1)

(44, 47, 64, 67, 67)


## Make a DataFrame object to hold the contents of the data set
[DataFrame help page](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.html)

* Just like with the pd.Series call, you specify the data, index labels (row labels in this case)
* In addition to row labels, you can also specify column labels (with 'columns')
* Can also specify data type (default is inferred)
* Like pd.Series you can ask for an independent copy of the data (copy=True) or you will get a view by default (i.e. copy=False)

In [7]:
# make the call to pd.DataFrames to create the DF - usage much like pd.Series
df = pd.DataFrame(data = neuron_data, index=neuron_labels, columns = ['resp1', 'resp2'])

# take a look at the output...
display(df)   # compare to print(df) - looks nicer with display thanks to iPython backend 

,resp1,resp2
Nrn0,44,9
Nrn1,47,83
Nrn2,64,21
Nrn3,67,36
Nrn4,67,87


In [8]:
# another handy display function...good for large dfs that are too big to fit - at least you can get an idea of 
# the overall structure
df.head()

,resp1,resp2
Nrn0,44,9
Nrn1,47,83
Nrn2,64,21
Nrn3,67,36
Nrn4,67,87


## Get a high-level summary of the data using built-in functionality of DataFrame object
[API reference page](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html)

In [9]:
# first call this using the defaults
df.describe()

,resp1,resp2
count,5.000000,5.000000
mean,57.800000,47.200000
std,11.344602,35.835736
min,44.000000,9.000000
25%,47.000000,21.000000
50%,64.000000,36.000000
75%,67.000000,83.000000
max,67.000000,87.000000


### Can do more sophisticated summaries of the data with "describe" as well

In [10]:
# can specify different values to change behavior...
df.describe(percentiles=np.linspace(0,1,11))

,resp1,resp2
count,5.000000,5.000000
mean,57.800000,47.200000
std,11.344602,35.835736
min,44.000000,9.000000
0%,44.000000,9.000000
10%,45.200000,13.800000
20%,46.400000,18.600000
30%,50.400000,24.000000
40%,57.200000,30.000000
50%,64.000000,36.000000


## Indexing, adding, deleting entire columns
* Think of the DF as a dict of Series objects with common labels - much of the syntax is the same as for dicts (and for Series)

In [11]:
# grab the second column from our DF
display(df['resp2'])

Nrn0     9
Nrn1    83
Nrn2    21
Nrn3    36
Nrn4    87
Name: resp2, dtype: int64

### Adding a column is easy and can be done dynamically (on the fly)

In [12]:
# define a third response column as the product of the first two columns
df['resp3'] = df.resp1 * df.resp2
display(df)

,resp1,resp2,resp3
Nrn0,44,9,396
Nrn1,47,83,3901
Nrn2,64,21,1344
Nrn3,67,36,2412
Nrn4,67,87,5829


### Removing columns is also easy and done on the fly... 

In [13]:
# using the del command will delete a column from the DF
# note that here you have to use the df['resp3'] notation
# the df.resp3 notation will not work.
del df['resp3']

### Instead of deleting outright, you can also "pop" a column out and asign it to another variable

In [14]:
# define a third response column as the product of the first two columns
df['resp3'] = df.resp1 * df.resp2

# then pop it out
resp3 = df.pop('resp3')
display(resp3)

# now df is back down to just 2 columns
display(df)

Nrn0     396
Nrn1    3901
Nrn2    1344
Nrn3    2412
Nrn4    5829
Name: resp3, dtype: int64

,resp1,resp2
Nrn0,44,9
Nrn1,47,83
Nrn2,64,21
Nrn3,67,36
Nrn4,67,87


## More on indexing and selection of specific coordinates in a DF

### Row selection - this is a bit more confusing as there are many methods
* You can use df.loc to select a row by its label name
* You can use df.iloc to select a row by its integer location 
* You can use boolean vectors to select a set of rows that satisfy some condition
* You can slice rows using standard notation e.g. df[1:3] for rows 1-3
* You can also isolate a particular row/column using a combo of column indexing (see above) and standard slicing notation

<div class="alert alert-danger">
Contrary to usual slicing conventions, both the start and the stop indices are included when using the DF.LOC function...see below for demo. This makes sense because you're indexing by label name, not by a zero-based integer index. 
</div>

In [15]:
# data from 2nd neuron across both stimulus conditions
df.loc['Nrn1']

resp1    47
resp2    83
Name: Nrn1, dtype: int64

In [16]:
# CAREFUL!
# data from 2nd-6th neuron inclusive across both stimulus conditions
df.loc['Nrn1':'Nrn5']

# again, just need to be careful but this makes sense given that you're indexing based on label name (not 0-based counting)

,resp1,resp2
Nrn1,47,83
Nrn2,64,21
Nrn3,67,36
Nrn4,67,87


In [17]:
# data from 5th neuron across both stimulus conditions
df.iloc[4]

resp1    67
resp2    87
Name: Nrn4, dtype: int64

In [18]:
# data from 4th neuron, resp1 only
df.iloc[3][1]

36

In [19]:
# can use the trick for returning only a subset of values from a function:
# here grab the 2nd entry from the 2nd column
print('2nd column, 2nd entry')
print(df['resp2'][1])

# can also go like this
print('2nd column, 4th entry')
print(df.resp2[3])

2nd column, 2nd entry
83
2nd column, 4th entry
36


## Just as with the Series objects, we can use most NumPy operations on DFs without any trouble 

In [20]:
# true matrix mulitplication (not element by element)
df1 = df
np.dot(df, df1.T)

array([[ 2017,  2815,  3005,  3272,  3731],
       [ 2815,  9098,  4751,  6137, 10370],
       [ 3005,  4751,  4537,  5044,  6115],
       [ 3272,  6137,  5044,  5785,  7621],
       [ 3731, 10370,  6115,  7621, 12058]])

## And also just like the Series object, you can also apply many other standard analyses using the DF object directly

* [Pandas doc for all functions](https://pandas.pydata.org/pandas-docs/stable/api.html#api-dataframe-stats)

In [21]:
df.mean(axis=0)

resp1    57.8
resp2    47.2
dtype: float64

## Given our main DF with the data from the simulated unit recording study, demonstrate some simple file I/O
* Start by saving the data in a csv (comma separated values) file, which is a common format that is useful for small-to-medium sized data and can easily store mixed data types (strs, ints, floats, etc)

[more than you ever wanted to know about the CSV format](https://www.loc.gov/preservation/digital/formats/fdd/fdd000323.shtml)

[API reference page for csv read/write in Pandas](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html)

In [22]:
# Check out API reference page for all the flags, but for the moment
# the we'll save our header in the file so that reading it back in is easier...
# can do this either way, you just need to know how a file was written when 
# go to read it 
df.to_csv('spike_rates.csv',index=True,header=True)

### Have a look at the csv file in a text editor...

In [23]:
# use our current working directory to build a path to the file
print(cwd)
file_name = cwd + '/spike_rates.csv'
print(file_name)

# read back in the .csv file that we made above. 
# by default read_csv will try to infer the column headers but since
# we explicitly wrote them out (header=True on write) we can tell 
# read_csv that row 0 in the file has the header info...
# same for the row labels (index labels)...tell
# read_csv that our row labels are in the first column (0) 
# of the file
df = pd.read_csv(file_name, index_col=0, header=0)
display(df)

/Users/johnserences/Dropbox/teaching/NEUG-PSYC 231 Fall 2021/11292021_Pandas
/Users/johnserences/Dropbox/teaching/NEUG-PSYC 231 Fall 2021/11292021_Pandas/spike_rates.csv


,resp1,resp2
Nrn0,44,9
Nrn1,47,83
Nrn2,64,21
Nrn3,67,36
Nrn4,67,87


## Making cooler DataFrame styles (and more useful...although that should take a backseat to coolness)
[Check here for a bunch of neat style options](https://pandas.pydata.org/pandas-docs/stable/style.html)
* Simple demo - can write custom functions that highlight specific aspects of your data - can be very useful for more clearly highlighting/communicating key points in the data within a notebook  

In [24]:
# highlight the max value in each column in yellow...
def highlight_max_value_in_columns(data_frame):
    ind_max = data_frame == data_frame.max()
    return ['background-color: yellow' if i else '' for i in ind_max]


In [25]:
# apply it here!
df.style.apply(highlight_max_value_in_columns)

,resp1,resp2
Nrn0,44,9
Nrn1,47,83
Nrn2,64,21
Nrn3,67,36
Nrn4,67,87
